# enVector Example with Encrypted Query

In this example, we will walk through the steps to use enVector with an encrypted query.

If data for the query also need to be secured, we can use CC (Cipher Query - Cipher Index) search. For example, to retrieve data from an index with restricted access, clients should verify that their encrypted security identifier is contained within the encrypted list of authorized privileges.

Note that query encryption is supported only by `eval_mode=rmp` during key setup. The advanced algorithm `mm` does not currently support ciphertext queries, as it's under development.

## Import SDK

To use the enVector SDK (`es2`), you need to install it first.
Before installing, make sure you have Python 3.12 and a virtual environment on your system.
After installation, you can import the SDK in your Python code.

In [ ]:
import es2

## Initialize

To use the enVector service, initialization is required. 

The following initialization step includes establishing a connection to the enVector server and configuring cryptographic settings necessary for vector search.

In [ ]:
es2.init(
    address="localhost:50050",
    # access_token="...", # if needed
    key_path="./keys",
    key_id="quickstart_key",
)

## Prepare Data

### Prepare Plaintext Vectors

Let's generate a random dataset first, which will become the secured, encrypted vector index.

In [ ]:
import numpy as np

vecs = np.random.rand(10, 512)
vecs = vecs / np.linalg.norm(vecs, axis=1, keepdims=True)
metadata = [f"Item {i+1}" for i in range(10)]

## Create Index and Insert Data

To use an encrypted query for search, we should configure the index to allow encrypted queries.
The query encryption mode supports `plain` and `cipher`.

In [ ]:
# Create index with query encryption config as cipher
index = es2.create_index(
    "quickstart_index_cc", 
    dim=512, 
    query_encryption="cipher"
)

# Or the following approach is also available after index creation.
# index.index_config.query_encryption = "cipher"

When the index is ready, you can insert data into it.
This first encrypts the vectors using the generated encryption keys internally in `index` and inserts them into the created index.

In [ ]:
index.insert(vecs, metadata=metadata)

## Encrypted Similarity Search

### Prepare query

First, prepare a query for encrypted search.
For now, let's use the first data point we generated randomly as the query.

In [ ]:
query_vector = vecs[0]

### Encrypted search on the index

Let's perform the encrypted similarity search with an encrypted query. 
Since we already configured the index to use encrypted queries, the plain query input will automatically be encrypted during the process.
This process ensures secure and efficient similarity search operations, even when working with encrypted data.

In [ ]:
# Encrypt query
# index.search() includes query encryption internally, 
# but here we show the explicit encryption step.
cipher = es2.Cipher("keys/quickstart_key/EncKey.bin", dim=512)
encrypted_query_vector = cipher.encrypt(query_vector, encode_type="query")

# Perform encrypted search
result = index.search(encrypted_query_vector, top_k=2, output_fields=["metadata"])[0]
result

### Clean Up

We can delete the created index and the registered key when they are no longer needed.

In [ ]:
es2.drop_index("quickstart_index_cc")

In [ ]:
es2.delete_key("quickstart_key")